## Importamos las librerias.

In [ ]:
import requests
import pandas as pd
from datetime import datetime

- Creamos la función **`get_historical_data`** para hacer la solicitud a la *API de Coingecko* y extraer la información de las monedas con mayor capitalización seleccionadas, por un período de 14 días, a partir del 01-08-2023 hasta el 15-08-2023.

In [4]:
def get_historical_data(id, days):
    info_endpoint = f"https://api.coingecko.com/api/v3/coins/{id}"
    info_response = requests.get(info_endpoint)
    
    if info_response.status_code == 200:
        coin_info = info_response.json()
        coin_name = coin_info["name"]
    else:
        print(f"Error al obtener la información de {id}. Detalles del error:")
        print(info_response.text)
        return None
    
    history_endpoint = f"https://api.coingecko.com/api/v3/coins/{id}/market_chart"
    history_params = {
        "vs_currency": "usd",
        "days": days,
        "interval": "daily"
    }
    
    history_response = requests.get(history_endpoint, params=history_params)
    
    if history_response.status_code == 200:
        history_data = history_response.json()
        timestamps = [entry[0] for entry in history_data["prices"]]
        prices = [entry[1] for entry in history_data["prices"]]
        market_caps = [entry[1] for entry in history_data["market_caps"]]
        total_volumes = [entry[1] for entry in history_data["total_volumes"]]
        
        dates = [datetime.fromtimestamp(ts / 1000).strftime('%Y-%m-%d') for ts in timestamps]
        
        df_historical = pd.DataFrame({
            "Date": dates,
            "Price": prices,
            "MarketCap": market_caps,
            "TotalVolume": total_volumes,
            "Currency": [coin_name] * len(timestamps)
        })
        
        return df_historical
    else:
        print(f"Error al obtener los datos históricos para {id}. Detalles del error:")
        print(history_response.text)
        return None

# Símbolos de las 10 monedas principales
top = ["bitcoin", "ethereum", "binancecoin", "ripple", "dogecoin"]

days = "14"  

- Creamos el DataFrame **df_top** y lo guardamos como un archivo csv llamado **top_historical_data.csv**

In [5]:
df_top = pd.DataFrame()
for symbol in top:
    df_symbol = get_historical_data(id=symbol, days=days)
    if df_symbol is not None:
        df_top = pd.concat([df_top, df_symbol], ignore_index=True)

df_top.to_csv("top_historical.csv", index=False)

- Verificamos el contnido del DataFrame.

In [6]:
df_top

,Date,Price,MarketCap,TotalVolume,Currency
0,2023-08-01,29537.109747,5.724089e+11,1.528992e+10,Bitcoin
1,2023-08-02,29146.636271,5.670747e+11,1.926632e+10,Bitcoin
2,2023-08-03,29176.150597,5.679406e+11,1.253913e+10,Bitcoin
3,2023-08-04,29087.938086,5.650731e+11,1.211287e+10,Bitcoin
4,2023-08-05,29046.800457,5.649701e+11,5.335309e+09,Bitcoin
...,...,...,...,...,...
70,2023-08-11,0.075714,1.063630e+10,2.637410e+08,Dogecoin
71,2023-08-12,0.076696,1.077445e+10,3.855159e+08,Dogecoin
72,2023-08-13,0.074778,1.050395e+10,3.255308e+08,Dogecoin
73,2023-08-14,0.074764,1.051512e+10,4.339586e+08,Dogecoin
